# Nuclio - Data preperation function

## Environment

In [1]:
# nuclio: ignore
import nuclio

### Configurations

In [2]:
%%nuclio config

# Trigger
spec.triggers.retrain.kind = "cron"
spec.triggers.retrain.attributes.interval = "1h"

# Base image
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.retrain.kind to 'cron'
%nuclio: setting spec.triggers.retrain.attributes.interval to '1h'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Commands

In [17]:
%%nuclio cmd -c

############
# installs #
############

# Utils
pip install pyarrow
pip install pandas

# Igz DB
pip install v3io_frames --upgrade

# Function
pip install dask["complete"]

### Variables

In [18]:
%%nuclio env

# DB Config
V3IO_FRAMESD=${V3IO_FRAMESD}
V3IO_USERNAME=${V3IO_USERNAME}
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

# Metrics
METRICS_TABLE=netops_metrics

# Features
FEATURES_TABLE=netops_features

# Parallelizem
NUMBER_OF_SHARDS=4

%nuclio: setting 'V3IO_FRAMESD' environment variable
%nuclio: setting 'V3IO_USERNAME' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'METRICS_TABLE' environment variable
%nuclio: setting 'FEATURES_TABLE' environment variable
%nuclio: setting 'NUMBER_OF_SHARDS' environment variable


%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line


## Function

### Imports

In [19]:
# Utils
import os
import time
import pandas as pd
import itertools

# DB Connection
import v3io_frames as v3f

# Parallelization
import dask.dataframe as dd
from dask.distributed import Client

### Helper functions

In [20]:
def normalize_timestamp(df):
#     df['timestamp'] = df['timestamp'].dt.strftime('%Y%m%d %H:%M')
    return df

In [47]:
def format_df_from_tsdb(context, df):
    df.index.names = ['timestamp', 'company', 'data_center', 'device']
    df = df.reset_index()
    df = normalize_timestamp(df)
    df = dd.from_pandas(df, npartitions=context.shards)
    return df

In [40]:
def get_data(context):
    df = context.v3f.read(backend='tsdb', query=f'select cpu_utilization, latency, packet_loss, throughput, is_error from {context.metrics_table}',
                          start=f'now-2h', end='now', multi_index=True)
    df = format_df_from_tsdb(context, df)
    return df

In [41]:
def create_rolling_featuers(context, df, window_size: int):
    features = df.copy()
    features['key'] = features.apply(lambda row: f'{row["company"]}_{row["data_center"]}_{row["device"]}', axis=1, meta=features.compute().dtypes)
    features.set_index('key')
    features["cpu_utilization"] = features.cpu_utilization.rolling(window=window_size).mean()
    features["latency"] = features.latency.rolling(window=window_size).mean()
    features["packet_loss"] = features.packet_loss.rolling(window=window_size).mean()
    features["throughput"] = features.throughput.rolling(window=window_size).mean()
    features["is_error"] = features.is_error.rolling(window=window_size).max()
                                     
    features = features.dropna()
    features = features.drop_duplicates()

    return features

In [42]:
def save_to_tsdb(context, features: pd.DataFrame):   
    context.v3f.write('tsdb', context.features_table, features)

### Init context

In [43]:
def init_context(context):
    
    # Create our DB client
    v3io_client = v3f.Client(address='http://' + os.getenv('V3IO_FRAMESD', 'framesd:8081'), 
                        container='bigdata', 
                        password=os.environ['V3IO_ACCESS_KEY'], 
                        user=os.environ['V3IO_USERNAME'])
    setattr(context, 'v3f', v3io_client)
    
    # Create Dask client
    dask_client = Client()
    setattr(context, 'dask', dask_client)
    
    # Set time to train on
    train_on_last = os.getenv('TRAIN_ON_LAST', '7d')
    setattr(context, 'train_on_last', train_on_last)
    
    # Set training set size
    train_set_size = float(os.getenv('TRAIN_SIZE', 0.7))
    setattr(context, 'train_size', train_set_size)
    
    # Netops metrics table
    setattr(context, 'metrics_table', os.getenv('METRICS_TABLE', 'netops_metrics'))
    
    # Netops feautres table
    setattr(context, 'features_table', os.getenv('FEATURES_TABLE', 'netops_features'))
    context.v3f.create('tsdb', context.features_table, attrs={'rate': '1/s'}, if_exists=1)
    
    # Dask shards / CV
    setattr(context, 'shards', int(os.getenv('NUMBER_OF_SHARDS', 4)))

### Handler

In [44]:
def handler(context, event):
    
    # Get data
    raw = get_data(context) 
    
    # Get minute features
    minute = create_rolling_featuers(context, raw, 3)
    
    # Get hour features
    hour = create_rolling_featuers(context, raw, 3*60)
    column_names = {'cpu_utilization': 'cpu_utilization_hourly',
                    'latency': 'latency_hourly',
                    'packet_loss': 'packet_loss_hourly',
                    'throughput': 'throughput_hourly'}
    hour = hour.rename(columns=column_names)
    
    # Create feature vector from data sources
    features_rm = raw.merge(minute, on=['timestamp', 'company', 'data_center', 'device'], suffixes=('_raw', '_minute'))
    features_rm.compute()
    
    features = features_rm.merge(hour, on=['timestamp', 'company', 'data_center', 'device'], suffixes=('_raw', '_hourly'))
    features = features.compute()
    
    # Save feature vector to TSDB
    
    # Drop key columns
    features = features.reset_index(drop=True)
    feature_cols = [col for col in features.columns if 'key' in col]
    features = features.drop(feature_cols, axis=1)
    
    
    # Fix indexes before saving
    features = features.set_index(['timestamp', 'company', 'data_center', 'device'])
    print(features)
    
    # Save to TSDB
    save_to_tsdb(context, features)

## Test

In [45]:
# nuclio: ignore
init_context(context)

/conda/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [48]:
# nuclio: ignore
# init_context(context)
event = nuclio.Event(body='')
output = handler(context, event)
output

                                                                   cpu_utilization_raw  \
timestamp           company         data_center   device                                 
2019-05-15 05:33:22 Alvarez_Cliff   1443091621871 Hayes-Alexander            93.847120   
2019-05-15 05:33:27 Alvarez_Cliff   1443091621871 Hayes-Alexander            63.889909   
2019-05-15 05:33:32 Alvarez_Cliff   1443091621871 Hayes-Alexander            71.950348   
2019-05-15 05:33:37 Alvarez_Cliff   1443091621871 Hayes-Alexander            81.903400   
2019-05-15 05:33:42 Alvarez_Cliff   1443091621871 Hayes-Alexander            60.637442   
2019-05-15 05:33:47 Alvarez_Cliff   1443091621871 Hayes-Alexander            78.055295   
2019-05-15 05:33:52 Alvarez_Cliff   1443091621871 Hayes-Alexander            49.095087   
2019-05-15 05:33:57 Alvarez_Cliff   1443091621871 Hayes-Alexander            75.828966   
2019-05-15 05:34:02 Alvarez_Cliff   1443091621871 Hayes-Alexander            80.084652   
2019-05-15

## Deployment

In [49]:
%nuclio deploy -p netops -n PreProcessing -c

%nuclio: ['deploy', '-p', 'netops', '-n', 'PreProcessing', '-c', '/User/netops/tutorials/demos/netops/Nuclio-Data_Preperation.ipynb']
%nuclio: [nuclio.deploy] 2019-05-15 07:18:57,755 (info) Building processor image
%nuclio: [nuclio.deploy] 2019-05-15 07:19:02,802 (info) Pushing image
%nuclio: [nuclio.deploy] 2019-05-15 07:19:02,802 (info) Build complete
%nuclio: [nuclio.deploy] 2019-05-15 07:19:06,847 (info) Function deploy complete
%nuclio: [nuclio.deploy] 2019-05-15 07:19:06,853 done updating preprocessing, function address: 3.122.56.83:30006
%nuclio: function deployed
